In [7]:
from elasticsearch import Elasticsearch
import pandas as pd
from elasticsearch.helpers import bulk

In [5]:
# Elasticsearch 클라이언트 생성
es = Elasticsearch("http://localhost:9200")  # 로컬에서 실행되는 Elasticsearch에 연결

# 연결 확인
if es.ping():
    print("Connected to Elasticsearch")
else:
    print("Elasticsearch connection failed")


Connected to Elasticsearch


In [6]:
# 인덱스 이름 설정
index_name = "products"

# 1. 인덱스 생성
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)
    print(f"Index '{index_name}' created")
else:
    print(f"Index '{index_name}' already exists")

Index 'products' already exists


In [10]:
# 2. CSV 파일 읽기
csv_file = "C:/Users/user/Desktop/캡스톤/filtered_mart_data3.csv"  # CSV 파일 경로
df = pd.read_csv(csv_file)

# 3. 데이터 준비 및 업로드
def generate_data(df, index_name):
    for i, row in df.iterrows():
        yield {
            "_index": index_name,
            "_id": i + 1,  # 문서 ID 설정
            "_source": {
                "category": row["카테고리"],
                "keyword": row["키워드"],
                "image_url": row["이미지 URL"],
                "product_name": row["상품명"],
                "discount_price": int(row["할인가"]),
                "original_price": int(row["원래 가격"]),
                "title": row["제목"],
                "description": row["설명"],
                "is_discounted": row["할인가격 여부"]
            }
        }

# Bulk 업로드 실행
bulk(es, generate_data(df, index_name))
print(f"Uploaded {len(df)} documents to index '{index_name}'")

Uploaded 551 documents to index 'products'


In [13]:
# res = es.search(index="products", body={"query": {"match_all": {}}}, size=10)
# for doc in res['hits']['hits']:
#     print(doc['_source'])


In [14]:
# res = es.search(index="products", body={
#     "query": {
#         "match": {
#             "category": "정육"
#         }
#     }
# })
# for doc in res['hits']['hits']:
#     print(doc['_source'])